In [1]:
import os
os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages io.delta:delta-core_2.12:1.2.1,org.apache.spark:spark-sql-kafka-0-10_2.12:3.5.0 pyspark-shell'

In [ ]:
#!pip install delta-spark

In [3]:
pip install delta-spark==3.3.1

ERROR: Could not find a version that satisfies the requirement delta-spark==3.3.1 (from versions: 0.0.1, 1.0.0, 1.0.1, 1.1.0, 1.2.0, 1.2.1, 2.0.0rc1, 2.0.0, 2.0.1, 2.0.2, 2.1.0, 2.1.1, 2.2.0, 2.3.0, 2.4.0, 3.0.0rc1, 3.0.0, 3.1.0, 3.2.0, 3.2.1, 3.3.0, 4.0.0rc1)
ERROR: No matching distribution found for delta-spark==3.3.1
Note: you may need to restart the kernel to use updated packages.


In [4]:
from pyspark.sql import SparkSession
from pyspark.sql.types import *
import pyspark.sql.functions as F
from delta import *
import pyspark

spark = pyspark.sql.SparkSession.builder.appName('Stream Demo') \
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog")

spark = configure_spark_with_delta_pip(spark).getOrCreate()

# Set the legacy time parser policy to handle the date format correctly
spark.conf.set("spark.sql.legacy.timeParserPolicy", "LEGACY")

spark 

In [5]:
from pyspark.sql.functions import *

weather_stream = spark \
    .readStream \
    .format("kafka") \
    .option("kafka.bootstrap.servers", "kafka:9092") \
    .option("subscribe", "weather-data") \
    .option("startingOffsets", "latest") \
    .load()

#.option("startingOffsets", "latest") \

#.option("startingOffsets", "latest") \


traffic_stream = spark \
    .readStream \
    .format("kafka") \
    .option("kafka.bootstrap.servers", "kafka:9092") \
    .option("subscribe", "traffic-data") \
    .option("startingOffsets", "latest") \
    .load()


#.option("startingOffsets", "latest") \


weather_json_df = weather_stream.selectExpr("CAST(value AS STRING) as value")

traffic_json_df = traffic_stream.selectExpr("CAST(value AS STRING) as value")

#traffic_json_df = traffic_stream.withColumn('value', expr('cast(value as string)')).withColumn('key', expr('cast(key as string)'))

# {"latitude": 49.2838889, "longitude": -122.7933334, "current_speed": 28, "free_flow_speed": 28, 
#"current_travel_time": 131, "free_flow_travel_time": 131, "confidence": 1, "road_closure": false}

weather_schema = StructType([
    StructField('name', StringType()),
    StructField('latitude', DoubleType()),
    StructField('longitude', DoubleType()),
    StructField('date', IntegerType()),
    StructField('weather', StringType()),
    StructField('weather_description', StringType()),
    StructField('temp', DoubleType()),
    StructField('visibility', IntegerType()),
    StructField('clouds', IntegerType()),
    StructField('rain', DoubleType()),
    StructField('snow', DoubleType()),
])

traffic_schema = StructType([
    StructField("latitude", DoubleType(), True),
    StructField("longitude", DoubleType(), True),
    StructField("current_speed", IntegerType(), True),
    StructField("free_flow_speed", IntegerType(), True),
    StructField("current_travel_time", IntegerType(), True),
    StructField("free_flow_travel_time", IntegerType(), True),
    StructField("confidence", IntegerType(), True),
    StructField("road_closure", BooleanType(), True),
    StructField("date", StringType(), True)
])

# Parse the 'value' column as JSON
weather_parsed_df = weather_json_df.select(from_json("value", weather_schema).alias("data"))
traffic_parsed_df = traffic_json_df.select(from_json("value", traffic_schema).alias("data"))
#traffic_parsed_df = traffic_json_df.withColumn("values_json", from_json(col("value"), traffic_schema))


# Flatten the JSON into separate columns
weather_flatten_df = weather_parsed_df.select(
                col('data.name').alias('name'),
                col('data.latitude').alias('latitude'),
                col('data.longitude').alias('longitude'), 
                from_unixtime(col('data.date')).alias('date_unix'),
                from_utc_timestamp(col("date_unix"), "America/Los_Angeles").alias("date"),
                col('data.weather').alias('weather'), 
                col('data.weather_description').alias('weather_description'), 
                col('data.temp').alias('temp'), 
                col('data.visibility').alias('visibility'),
                col('data.clouds').alias('clouds'),
                col('data.rain').alias('rain'),
                col('data.snow').alias('snow'))
    
weather_flatten_df = weather_flatten_df.drop("date_unix")

traffic_flatten_df = traffic_parsed_df.select(
    col("data.latitude").alias("latitude"), 
    col("data.longitude").alias("longitude"),
    col("data.current_speed").alias("current_speed"),
    col("data.free_flow_speed").alias("free_flow_speed"),
    col("data.current_travel_time").alias("current_travel_time"),
    col("data.free_flow_travel_time").alias("free_flow_travel_time"),
    col("data.confidence").alias("confidence"),
    col("data.road_closure").alias("road_closure"),
    to_timestamp(col("data.date"), "EEE, dd MMM yyyy HH:mm:ss z").alias("date_utc"),
    from_utc_timestamp(col("date_utc"), "America/Los_Angeles").alias("date")
)

traffic_flatten_df = traffic_flatten_df.drop("date_utc")

#traffic_flatten_df.show(truncate=False)
#weather_parsed_df.printSchema()
#weather_flatten_df.show(truncate=False)

# Convert date string to timestamp in UTC
#to_timestamp(col("values_json.date"), "EEE, dd MMM yyyy HH:mm:ss z").alias("date_utc"),
# Convert to PST
#from_utc_timestamp(col("values_json.date"), "America/Los_Angeles").alias("date")


# Convert UNIX timestamp to 
#weather_flatten_df = weather_flatten_df.withColumn((from_unixtime(col("date")).alias("date"))
#traffic_flatten_df.show(truncate=False)
#traffic_flatten_df.printSchema()

In [6]:
from pyspark.sql.functions import col

# Set the interval (e.g., hourly) for the trigger
trigger_interval = "1 hour" 

weather_query = weather_flatten_df \
    .writeStream \
    .outputMode('append') \
    .format('delta') \
    .option("checkpointLocation", "s3://van-crash-data/checkpoints/weather") \
    .trigger(processingTime=trigger_interval) \
    .start("s3://van-crash-data/weather-data") 

traffic_query = traffic_flatten_df \
    .writeStream \
    .outputMode('append') \
    .format('delta') \
    .option("checkpointLocation", "s3://van-crash-data/checkpoints/traffic") \
    .trigger(processingTime=trigger_interval) \
    .start("s3://van-crash-data/traffic-data")

print("Streaming started... waiting for data...")

weather_query.awaitTermination(60)
traffic_query.awaitTermination(60)

Py4JJavaError: An error occurred while calling o134.start.
: org.apache.hadoop.fs.UnsupportedFileSystemException: No FileSystem for scheme "s3"
	at org.apache.hadoop.fs.FileSystem.getFileSystemClass(FileSystem.java:3443)
	at org.apache.hadoop.fs.FileSystem.createFileSystem(FileSystem.java:3466)
	at org.apache.hadoop.fs.FileSystem.access$300(FileSystem.java:174)
	at org.apache.hadoop.fs.FileSystem$Cache.getInternal(FileSystem.java:3574)
	at org.apache.hadoop.fs.FileSystem$Cache.get(FileSystem.java:3521)
	at org.apache.hadoop.fs.FileSystem.get(FileSystem.java:540)
	at org.apache.hadoop.fs.Path.getFileSystem(Path.java:365)
	at org.apache.spark.sql.delta.DeltaLog$.apply(DeltaLog.scala:560)
	at org.apache.spark.sql.delta.DeltaLog$.apply(DeltaLog.scala:540)
	at org.apache.spark.sql.delta.DeltaLog$.forTable(DeltaLog.scala:482)
	at org.apache.spark.sql.delta.sources.DeltaSink.<init>(DeltaSink.scala:46)
	at org.apache.spark.sql.delta.sources.DeltaDataSource.createSink(DeltaDataSource.scala:134)

In [ ]:
weather_query.stop()
traffic_query.stop()